In [1]:
import os
import glob
import gzip
import math
import random
import pickle

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
from matplotlib.colors import LogNorm
import shapely.wkt as wkt
from shapely.geometry import Point, LineString, box
from shapely.ops import nearest_points
import lxml.etree as ET
import tqdm
import wandb
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as T
from torch.utils.data import DataLoader, Dataset, Subset
import torch_geometric
from torch_geometric.data import Data, Batch
from torch_geometric.transforms import LineGraph

import network_io as nio
import processing_io as pio

# Parameters to define
is_for_1pm = True
result_path = "dataset_1pct_0_100"

if is_for_1pm:
    base_dir = '../../../../data/pop_1pm_with_policies/'
    subdirs_pattern = os.path.join(base_dir, 'output_networks_*')
    subdirs = list(set(glob.glob(subdirs_pattern)))
    subdirs.sort()
    base_output_links_no_policies = '../../../data/pop_1pm/output_1pm/'
else:
    base_path = '../../../../data/pop_1pct_with_policies/output_networks_100/'
    network_folders = glob.glob(os.path.join(base_path, 'network_d_*'))
    base_output_links_no_policies = '../../../../data/pop_1pct/output_1pct/'

## Process results

Process the outputs of the simulations for further usage by GNN.

In [2]:
# Read all network data into a dictionary of GeoDataFrames
result_dic = {}
result_dic_single_districts = {}

if is_for_1pm:
    # Loop through each subdirectory
    for subdir in subdirs:
        print(f'Accessing folder: {subdir}')
        print(len(os.listdir(subdir)))
        for network in os.listdir(subdir):
            folder = os.path.join(subdir, network)
            file_path = os.path.join(subdir, network)
            if file_path.endswith(".DS_Store"):
                continue
            policy_key = pio.create_policy_key_1pm(network)
            gdf = nio.read_network_data(file_path)
            if gdf is not None:
                result_dic[policy_key] = gdf
else:
    for folder in network_folders:
        policy_key = pio.create_policy_key_1pct(os.path.basename(folder))
        try:
            gdf = nio.read_network_data(folder)
        except Exception:
            print(f"Error reading folder: {folder}")
        
        if gdf is not None:
            result_dic[policy_key] = gdf
        is_single  = pio.is_single_district(os.path.basename(folder))
        if is_single:
            result_dic_single_districts[policy_key] = gdf
        
base_network_no_policies = nio.read_network_data(base_output_links_no_policies)
result_dic["base_network_no_policies"] = base_network_no_policies

Error reading folder: ../../../../data/pop_1pct_with_policies/output_networks_100/network_d_20


## Analyze results and plot

In [3]:
for key, df in result_dic_single_districts.items():
    pio.plot_simulation_output(key, df)
    break

# pio.analyze_geodataframes(result_dic)

Policy: policy introduced in Arrondissement(s) 2, 4, 5, 6, 9, 11, 13, 15, 16
Total capacity of edges with 'car' mode: 143418
Percentage increase in 'vol_car': 0.06%
Percentage increase in capacity (car edges): -91.19%
Policy: policy introduced in Arrondissement(s) 1, 2, 3, 4, 5, 6, 9, 11, 14, 17, 18, 19
Total capacity of edges with 'car' mode: 144584
Percentage increase in 'vol_car': 0.86%
Percentage increase in capacity (car edges): -91.12%
Policy: policy introduced in Arrondissement(s) 1, 3, 4, 6, 9, 10, 11, 12, 15, 16, 18, 19, 20
Total capacity of edges with 'car' mode: 134571
Percentage increase in 'vol_car': 0.19%
Percentage increase in capacity (car edges): -91.73%
Policy: policy introduced in Arrondissement(s) 1, 2, 3, 5, 6, 7, 12, 15, 18, 19, 20
Total capacity of edges with 'car' mode: 141878
Percentage increase in 'vol_car': -0.01%
Percentage increase in capacity (car edges): -91.28%
Policy: policy introduced in Arrondissement(s) 3, 4, 6, 7, 9, 10, 11, 13, 14, 15, 16, 17, 18, 

## Save for further processing with GNN

In [4]:
data_processed = pio.process_result_dic(result_dic)
# data_processed_single_districts = pio.process_result_dic(result_dic_single_districts)

# torch.save(data_processed, result_path + '.pt')
# torch.save(data_processed_single_districts, result_path + '_single_districts.pt')